In [ ]:
%cd ..

In [ ]:
import glob
import os
from datetime import datetime

import polars as pl
from plotly.graph_objects import Figure

logs = glob.glob("logs/benchmark-amip/*.log")

dates = []
sypds = []

for log in logs:
    date = datetime.fromisoformat(os.path.basename(log).split(".")[0])
    dates.append(date)
    with open(log, "r") as f:
        lines = f.readlines()
    sypd = None
    found_step_40 = False
    for line in lines:
        if (
            "n_steps_completed = 40" in line
            or "n_steps_completed = 41" in line
        ):
            found_step_40 = True
        if found_step_40 and "estimated_sypd = " in line:
            sypd = float(
                line.split("estimated_sypd = ")[1].split()[0].replace('"', "")
            )
            break
    sypds.append(sypd)

df = pl.DataFrame({"date": dates, "sypd": sypds})
df = df.sort("date")

# Add normalized SYPD column; normalize by the first non-null SYPD value
first_sypd = df["sypd"].drop_nulls()[0]
df = df.with_columns(
    (pl.col("sypd") / first_sypd).alias("sypd_norm")
)

print(df)

os.makedirs("results", exist_ok=True)
df.write_csv("results/amip.csv")

fig: Figure = df.to_pandas().plot(
    x="date",
    y="sypd_norm",
    title="Flagship AMIP benchmark (16 horiz. elem.)",
    backend="plotly",
)  # type: ignore

# Use lines and markers
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis_title="Normalized SYPD", xaxis_title="Date")
fig.show()

# Save as PNG and JSON
os.makedirs("figures", exist_ok=True)
fig.write_image("figures/amip-sypd.png")
fig.write_json("figures/amip-sypd.json")